In [1]:
%reset -f
%matplotlib qt

import cv2
import numpy as np
import matplotlib.pyplot as plt
import math

In [2]:
# Carregando a imagem de teste
img = plt.imread('imgs/img5.jpeg')

## convertendo RGB para HSV
hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

##mascara verde
mask = cv2.inRange(hsv, (36, 25, 25), (86, 255,160))

##Separando a cor verde da imagem
imask = mask>0
green = np.zeros_like(img, np.uint8)
green[imask] = img[imask]

#deixando a parte verde branca
h,w,_ = green.shape
for e in range(h):
    for i in range(w):
        if green[e][i][0]>0 and green[e][i][1]>0 and green[e][i][1]>0:
            green[e][i]=[255, 255, 255]
            
plt.imshow(green)

In [109]:
#erosão e dilatação para remover ruidos
kernel = np.ones((5,5),np.uint8)
erosion = cv2.erode(green,kernel,iterations = 4)
dilation = cv2.dilate(erosion,kernel,iterations = 4)
plt.imshow(dilation)
plt.show()

In [4]:
#adicionando blur na imagem
# kernel2 = np.ones((5,5),np.float32)/25
# blur = cv2.bilateralFilter(dilation, 9, 75, 75)
# filter2d

# plt.imshow(np.uint32(blur))
# plt.show()

In [110]:
#adicionando blur na imagem usando gaussian blur
img_blur = cv2.GaussianBlur(dilation,(11,11),cv2.BORDER_DEFAULT)
plt.imshow(img_blur)
plt.show()

In [113]:
# Canny para detecção de bordas
edges = cv2.Canny(img_blur, 60, 120,apertureSize = 15)
plt.imshow(edges,cmap = 'gray')

error: OpenCV(4.1.2) /io/opencv/modules/imgproc/src/canny.cpp:846: error: (-206:Bad flag (parameter or structure field)) Aperture size should be odd between 3 and 7 in function 'Canny'


In [33]:
# compute the median of the single channel pixel intensities
sigma=0.33
v = np.median(img_blur)
 
# apply automatic Canny edge detection using the computed median
lower = int(max(0, (1.0 - sigma) * v))
upper = int(min(255, (1.0 + sigma) * v))
edged = cv2.Canny(img_blur, lower, upper)
plt.imshow(edged,cmap = 'gray')
print(lower, upper)

170 255


In [105]:
# Copy edges to the images that will display the results in BGR
cdst = cv2.cvtColor(edges, cv2.COLOR_GRAY2BGR)
cdstP_green = np.copy(cdst)

img_edge = np.zeros_like(img, np.uint8)

linesP = cv2.HoughLinesP(edges, 1, np.pi / 180, 50, None, 20, 10)
    
if linesP is not None:
    for i in range(0, len(linesP)):
        l = linesP[i][0]
        cv2.line(img_edge, (l[0], l[1]), (l[2], l[3]), (0,0,255), 3, cv2.LINE_AA)

plt.imshow(img_edge)        

In [114]:
#Novos valores!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!1
# Copy edges to the images that will display the results in BGR
cdst = cv2.cvtColor(edges, cv2.COLOR_GRAY2BGR)
cdstP_green = np.copy(cdst)

img_edge = np.zeros_like(img, np.uint8)

linesP = cv2.HoughLinesP(edges, 0.8, 1*np.pi / 180, 60, None, 100, 80)
    
if linesP is not None:
    for i in range(0, len(linesP)):
        l = linesP[i][0]
        cv2.line(img_edge, (l[0], l[1]), (l[2], l[3]), (0,0,255), 3, cv2.LINE_AA)

plt.imshow(img_edge)       